In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from data_manager import *
from metadata import *

import matplotlib.pyplot as plt
%matplotlib inline

Setting environment for Gordon


In [2]:
from skimage.filters import threshold_otsu
from scipy.ndimage.morphology import distance_transform_edt
from skimage.feature import peak_local_max
from skimage.morphology import watershed, binary_dilation, disk
from skimage.segmentation import mark_boundaries, find_boundaries
from skimage.measure import regionprops, label
from sklearn.cluster import KMeans
from skimage.transform import rotate, resize

from annotation_utilities import *
from registration_utilities import *

In [3]:
stack = 'MD589'

In [21]:
min_blob_area = 10
max_blob_area = 10000

In [5]:
output_dir = create_if_not_exists('/home/yuncong/csd395/CSHL_cells_v2/detected_cells/' + stack)

In [7]:
sections_to_filenames = metadata_cache['sections_to_filenames'][stack]
first_bs_section, last_bs_section = metadata_cache['section_limits'][stack]

In [27]:
bar = show_progress_bar(first_bs_section, last_bs_section)

# for sec in [242]:
for sec in range(first_bs_section, last_bs_section, 50):
# sec = 242

    fn = sections_to_filenames[sec]

    bar.value = sec

    sys.stderr.write('Processing section: %03d\n' % sec)

    t = time.time()

    img = imread(DataManager.get_image_filepath(stack=stack, section=sec, resol='lossless', version='cropped'))
    im = rgb2gray(img)

    imsave('/home/yuncong/csd395/CSHL_cells_v2/detected_cells/MD589/%(fn)s/%(fn)s_gray.jpg' % {'fn':fn}, img_as_ubyte(im))

    sys.stderr.write('Load image: %.2f\n' % (time.time() - t) )

    t = time.time()

    mask_tb = DataManager.load_thumbnail_mask_v2(stack=stack, section=sec)
    mask = resize(mask_tb, img.shape[:2]) > .5

    sys.stderr.write('Load mask: %.2f\n' % (time.time() - t) )

    t = time.time()

    thresh = threshold_otsu(im)
    binary = im < thresh
    binary[~mask] = 0

    sys.stderr.write('threshold: %.2f\n' % (time.time() - t) )
    imsave('/home/yuncong/csd395/CSHL_cells_v2/detected_cells/MD589/%(fn)s/%(fn)s_otsu.jpg' % {'fn':fn}, binary.astype(np.uint8)*255)

    # plt.imshow(binary, cmap=plt.cm.gray);
    # plt.axis('off');
    # plt.title('binary');

    t = time.time()
    dt = distance_transform_edt(binary)
    sys.stderr.write('distance transform: %.2f\n' % (time.time() - t) )

    # plt.imshow(dt, cmap=plt.cm.gray);
    # plt.axis('off');
    # plt.title('distance tranform');

    t = time.time()
    local_maxi = peak_local_max(dt, labels=binary, footprint=np.ones((10, 10)), indices=False)
    sys.stderr.write('local max: %.2f\n' % (time.time() - t) )

    # plt.imshow(local_maxi, cmap=plt.cm.gray);
    # plt.axis('off');
    # plt.title('local maxima');

    t = time.time()
    markers = label(local_maxi)
    sys.stderr.write('label: %.2f\n' % (time.time() - t) )

    t = time.time()
    labels = watershed(-dt, markers, mask=binary)
    sys.stderr.write('watershed: %.2f\n' % (time.time() - t) )

    # plt.imshow(labels);
    # plt.axis('off');
    # plt.title('labels');

    t = time.time()
    props = regionprops(labels)
    sys.stderr.write('regionprops: %.2f\n' % (time.time() - t) )

#     plt.hist([p.area for p in props], bins=100);
#     plt.title('Area histograms');
#     plt.show();

    valid_blob_indices = [i for i, p in enumerate(props) if p.area > min_blob_area and p.area < max_blob_area]
    sys.stderr.write('%d blobs identified.\n' % len(valid_blob_indices))

    fn_output_dir = create_if_not_exists(os.path.join(output_dir, fn))

    t = time.time()

    valid_blob_coords = [props[i].coords for i in valid_blob_indices] # r,c
    valid_blob_orientations = np.array([props[i].orientation for i in valid_blob_indices])
    valid_blob_centroids = np.array([props[i].centroid for i in valid_blob_indices]) # r,c
    valid_blob_majorAxisLen = np.array([props[i].major_axis_length for i in valid_blob_indices])
    valid_blob_minorAxisLen = np.array([props[i].minor_axis_length for i in valid_blob_indices])

    #     valid_blob_bbox = [np.array(props[i].bbox) for i in valid_blob_indices]
    blob_contours = find_contour_points(labels, sample_every=1)
    valid_blob_contours = [blob_contours[i+1][0] for i in valid_blob_indices]
    #     valid_blob_contours = [cnts[0] for i, cnts in enumerate(valid_blob_contours)]

    pickle.dump(valid_blob_contours, open(fn_output_dir + '/%(fn)s_blobContours.pkl' % {'fn':fn}, 'w'))

    pickle.dump(valid_blob_coords, open(fn_output_dir + '/%(fn)s_blobCoords.pkl' % {'fn':fn}, 'w'))
    bp.pack_ndarray_file(valid_blob_orientations, fn_output_dir + '/%(fn)s_blobOrientations.bp' % {'fn':fn})
    bp.pack_ndarray_file(valid_blob_centroids, fn_output_dir + '/%(fn)s_blobCentroids.bp' % {'fn':fn})
    bp.pack_ndarray_file(valid_blob_majorAxisLen, fn_output_dir + '/%(fn)s_blobMajorAxisLen.bp' % {'fn':fn})
    bp.pack_ndarray_file(valid_blob_minorAxisLen, fn_output_dir + '/%(fn)s_blobMinorAxisLen.bp' % {'fn':fn})

    sys.stderr.write('Compute blob properties, save: %.2f\n' % (time.time() - t) )


    t = time.time()

    h, w = img.shape[:2]
    for cnt in valid_blob_contours:
        img[np.minimum(h-1, np.maximum(cnt[:,1], 0)), np.minimum(w-1, np.maximum(cnt[:,0], 0))] = (255,0,0)

    imsave('/home/yuncong/csd395/CSHL_cells_v2/detected_cells/MD589/%(fn)s/%(fn)s_detection_image.jpg' % {'fn':fn}, img)

    sys.stderr.write('Save image: %.2f\n' % (time.time() - t) )

Processing section: 092
Load image: 98.46
Load mask: 9.52
threshold: 4.50
distance transform: 21.45
local max: 15.72
label: 2.36
watershed: 40.30
regionprops: 2.85
117872 blobs identified.
Compute blob properties, save: 143.88
Save image: 20.66
Processing section: 142
Load image: 108.76
Load mask: 9.50
threshold: 4.74
distance transform: 21.44
local max: 15.68
label: 2.36
watershed: 34.21
regionprops: 2.93
115623 blobs identified.
Compute blob properties, save: 127.81
Save image: 20.60
Processing section: 192
Load image: 116.44
Load mask: 9.53
threshold: 3.94
distance transform: 21.56
local max: 15.93
label: 2.36
watershed: 46.84
regionprops: 3.40
175151 blobs identified.
Compute blob properties, save: 196.37
Save image: 22.70
Processing section: 242
Load image: 102.90
Load mask: 9.61
threshold: 3.89
distance transform: 21.46
local max: 15.83
label: 2.36
watershed: 39.86
regionprops: 3.91
154845 blobs identified.
Compute blob properties, save: 169.66
Save image: 21.72
Processing sectio